In [4]:
import os
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

from pyspark.sql.types import (

    StructType,
)
import requests
import os
import logging

In [5]:
import dotenv
dotenv.load_dotenv(dotenv.find_dotenv())

True

In [6]:

logger = logging.getLogger(__name__)
gcp_sa_key_path = os.environ.get("GOOGLE_APPLICATION_CREDENTIALS")
partition_key = "FullOrbisBasicDescriptorsPresent_v1.0.0|CC-MAIN-2024-38"
bucket_seed_nodes = "ascii-supply-chain-research-input"
bucket_cc_results = "ascii-supply-chain-research-results"
bucket_cc_dev_results = "ascii-supply-chain-research-dev-results"
sample_data = "ascii-supply-chain-research-sample-data"
num_executors = 40
total_executor_memory_gb = 900
gcs_connector_url = "https://repo1.maven.org/maven2/com/google/cloud/bigdataoss/gcs-connector/hadoop3-2.2.31/gcs-connector-hadoop3-2.2.31-shaded.jar"
packages = [
    "io.delta:delta-spark_2.12:3.2.0",
    "org.postgresql:postgresql:42.7.0",
    "org.apache.hadoop:hadoop-aws:3.3.4",
    "org.apache.spark:spark-hadoop-cloud_2.12:3.5.0",
    "com.johnsnowlabs.nlp:spark-nlp_2.12:5.3.3",
]
memory_per_executor_gb = int(total_executor_memory_gb / num_executors)
spark_cfg = {
    "spark.hadoop.fs.s3a.buffer.dir": "/data/raid5/data/sparktmp",
    "spark.driver.memory": "100G",
    "spark.driver.maxResultSize": "100G",
    "spark.executor.instances": str(num_executors),
    "spark.executor.memory": f"{memory_per_executor_gb}G",
    "spark.executor.memoryOverhead": f"{int(memory_per_executor_gb * 0.1)}G",
    "spark.executor.extraJavaOptions": "-XX:+UseG1GC -XX:InitiatingHeapOccupancyPercent=35 -XX:ConcGCThreads=4 -XX:ParallelGCThreads=4",
    "spark.driver.extraJavaOptions": "-XX:+UseG1GC -XX:InitiatingHeapOccupancyPercent=35 -XX:ConcGCThreads=4 -XX:ParallelGCThreads=4",
    "spark.dynamicAllocation.enabled": "true",
    "spark.dynamicAllocation.minExecutors": "4",
    "spark.dynamicAllocation.maxExecutors": str(num_executors),
    "spark.dynamicAllocation.initialExecutors": "4",
    "spark.sql.adaptive.enabled": "true",
    "spark.sql.adaptive.skewedJoin.enabled": "true",
    "spark.sql.cbo.enabled": "true",
    "spark.sql.cbo.joinReorder.enabled": "true",
    "spark.sql.cbo.starSchemaDetection": "true",
    "spark.sql.autoBroadcastJoinThreshold": "500MB",
    "spark.sql.statistics.histogram.enabled": "true",
    "spark.sql.execution.arrow.pyspark.enabled": "true",
    "spark.sql.shuffle.partitions": "800",
    "spark.jars.packages": ",".join(packages),
    "spark.jars": gcs_connector_url,    "spark.hadoop.fs.s3a.committer.name": "directory",
    "spark.hadoop.fs.s3a.fast.upload": "true",
    "spark.hadoop.fs.s3a.path.style.access": "true",
    "spark.hadoop.fs.s3a.impl": "org.apache.hadoop.fs.s3a.S3AFileSystem",
    "spark.hadoop.fs.s3a.fast.upload.buffer": "bytebuffer",
    "spark.hadoop.fs.s3a.fast.upload.active.blocks": "4",
    "spark.hadoop.fs.s3a.access.key": os.environ["ASCII_AWS_ACCESS_KEY_ID"],
    "spark.hadoop.fs.s3a.secret.key": os.environ["ASCII_AWS_SECRET_ACCESS_KEY"],
    
    "spark.hadoop.fs.gs.impl": "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem",
    "spark.hadoop.fs.AbstractFileSystem.gs.impl": "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS",    
    "spark.hadoop.google.cloud.auth.service.account.enable": "true",
    "spark.hadoop.google.cloud.auth.service.account.json.keyfile": gcp_sa_key_path,
    
    "spark.sql.parquet.compression.codec": "zstd",
    "spark.local.dir": "/data/raid5/data/sparktmp",
    "spark.eventLog.dir": "/data/raid5/data/sparktmp",
    "spark.history.fs.logDirectory": "/data/raid5/data/sparktmp",
    "spark.sql.parquet.columnarReaderBatchSize": "1000",  # This could be fine tuned
    "spark.sql.parquet.enableVectorizedReader": "false",  # This is cerainly needed
    "spark.sql.sources.bucketing.enabled": "false",
    "spark.network.timeout": "800s",
    "spark.executor.heartbeatInterval": "60s",
    "spark.sql.files.maxPartitionBytes": "128MB",
    "spark.sql.files.openCostInBytes": "128MB",
    "spark.memory.fraction": "0.8",
    "spark.memory.storageFraction": "0.3",
    "spark.rapids.sql.enabled": "true",
    "spark.sql.extensions": "io.delta.sql.DeltaSparkSessionExtension",
    "spark.sql.catalog.spark_catalog": "org.apache.spark.sql.delta.catalog.DeltaCatalog",
    "spark.sql.execution.arrow.pyspark.enabled": "true",
    "spark.sql.execution.arrow.maxRecordsPerBatch": "512",
    "spark.databricks.delta.retentionDurationCheck.enabled": "false",
    "spark.sql.sources.partitionOverwriteMode": "dynamic",
    }
builder = SparkSession.builder.appName("tu_wien_embedding").master(f"local[{num_executors}]")
for key, value in spark_cfg.items():
    builder = builder.config(key, value)
spark = builder.getOrCreate()

26/01/18 09:01:09 WARN Utils: Your hostname, utf resolves to a loopback address: 127.0.1.1; using 172.24.0.1 instead (on interface br-df65483b261a)
26/01/18 09:01:09 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /home/picatto/.ivy2/cache
The jars for the packages stored in: /home/picatto/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
org.postgresql#postgresql added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
org.apache.spark#spark-hadoop-cloud_2.12 added as a dependency
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-d17dc64a-bdc4-4e9b-b831-0e91d17877bc;1.0
	confs: [default]


:: loading settings :: url = jar:file:/home/picatto/ascii/gen-ai/.pixi/envs/basic-research/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found io.delta#delta-spark_2.12;3.2.0 in central
	found io.delta#delta-storage;3.2.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
	found org.postgresql#postgresql;42.7.0 in central
	found org.checkerframework#checker-qual;3.31.0 in central
	found org.apache.hadoop#hadoop-aws;3.3.4 in central
	found com.amazonaws#aws-java-sdk-bundle;1.12.262 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
	found org.apache.spark#spark-hadoop-cloud_2.12;3.5.0 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found org.xerial.snappy#snappy-java;1.1.10.3 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found com.google.cloud.bigdataoss#gcs-connector;hadoop3-2.2.14 in central
	found joda-time#joda-time;2.12.5 in central
	found com.fasterxml.jackson.core#jackson-

In [ ]:
df = spark.read \
    .option("basePath", "gs://gen-ai-tu/news/raw") \
    .parquet("gs://gen-ai-tu/news/raw/year=2025/month=12") \
    .filter(F.col("main_lang") == "en")

# Now 'year' and 'month' columns exist and you can write them
df.write.mode("overwrite").partitionBy("year", "month", "day").parquet('/data/raid5/data/picatto/ascii/news/raw')

26/01/18 07:28:32 WARN SharedInMemoryCache: Evicting cached table partition metadata from memory due to size constraints (spark.sql.hive.filesourcePartitionFileCacheSize = 262144000 bytes). This may impact query planning performance.


In [12]:
df = spark.read.format("parquet").load(f"/data/raid5/data/picatto/ascii/news/raw/year=2025/month=11/")
df = df.withColumnRenamed('filename', 'path').withColumnRenamed('tree', 'html').withColumn('year', F.lit(2025)).withColumn('month', F.lit(11)).select(
['uri', 'text', 'html', 'langs', 'confs', 'http_date', 'http_last_modified', 'http_charset', 'surt_uri', 'host', 'path', 'main_lang', 'year', 'month', 'day'])
df.printSchema()
df.write.mode("overwrite").partitionBy("year", "month", "day").parquet('/data/raid5/data/picatto/ascii/news/raw')

root
 |-- uri: string (nullable = true)
 |-- text: string (nullable = true)
 |-- html: string (nullable = true)
 |-- langs: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- confs: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- http_date: string (nullable = true)
 |-- http_last_modified: string (nullable = true)
 |-- http_charset: string (nullable = true)
 |-- surt_uri: string (nullable = true)
 |-- host: string (nullable = true)
 |-- path: string (nullable = true)
 |-- main_lang: string (nullable = true)
 |-- year: integer (nullable = false)
 |-- month: integer (nullable = false)
 |-- day: integer (nullable = true)



In [33]:
df = spark.read.format("parquet").load(f"/data/raid5/data/picatto/ascii/news/raw/").select("uri", "host", "html", "text", "year", "month", "day", "main_lang", "http_date")
df.show()

+--------------------+--------------------+--------------------+--------------------+----+-----+---+---------+--------------------+
|                 uri|                host|                html|                text|year|month|day|main_lang|           http_date|
+--------------------+--------------------+--------------------+--------------------+----+-----+---+---------+--------------------+
|https://www.shrop...|  com.shropshirestar|<!DOCTYPE html><h...|Local news you ca...|2025|   11| 26|       en|2025-11-26T15:45:...|
|https://www.goal....|            com.goal|<!DOCTYPE html><h...|Goal.com\n  • SCO...|2025|   11| 26|       en|2025-11-26T16:31:...|
|https://zeenews.i...|           com.india|<style type="text...|हिन्दीEnglishमराठ...|2025|   11| 26|       en|2025-11-26T16:31:...|
|https://kz.kinoaf...|     info.kinoafisha|<!DOCTYPE html>\n...|Menu\nKinoafisha\...|2025|   11| 26|       en|2025-11-26T16:31:...|
|https://www.rawpi...|        com.rawpixel|<!DOCTYPE html><h...|rawpixel\nAl

## Text preprocessing

In [18]:
from typing import List, Optional, Tuple, Dict
from dataclasses import dataclass, field
import re
import hashlib

In [9]:
config ={
    'input_path': '/data/raid5/data/picatto/ascii/news/raw/',
    'sample_fraction': 0.1
}

In [10]:
def extract_text_with_docling(html: str) -> Optional[str]:
    """
    Extract clean text from HTML using Docling
    Falls back to trafilatura if Docling fails
    """
    if not html or not html.strip():
        return None
    
    try:
        from docling.document_converter import DocumentConverter
        from docling.datamodel.base_models import InputFormat
        
        converter = DocumentConverter(
            allowed_formats=[InputFormat.HTML]
        )
        
        # Convert HTML string
        result = converter.convert_string(html, input_format=InputFormat.HTML)
        
        if result and result.document:
            # Export to plain text (or markdown)
            text = result.document.export_to_text()
            return text.strip() if text else None
            
    except Exception as e:
        logger.debug(f"Docling extraction failed: {e}, falling back to trafilatura")
    
    # Fallback to trafilatura
    try:
        from trafilatura import extract
        text = extract(html, include_comments=False, include_tables=True)
        return text.strip() if text else None
    except Exception as e:
        logger.debug(f"Trafilatura extraction also failed: {e}")
    
    return None

In [30]:
small = df.limit(10).toPandas()

In [32]:
small['extracted_text'] = small.apply(
    lambda row: extract_text_with_docling(row['html']),
    axis=1
)
small

,uri,html,text,year,month,day,main_lang,extracted_text
0,https://www.shropshirestar.com/uk-news/2025/11...,"<!DOCTYPE html><html lang=""en"" class=""__variab...",Local news you can trust\nShropshire Star\nNew...,2025,11,26,en,Food price hike fears mount after supermarkets...
1,https://www.goal.com/en-in/lists/harry-kane-no...,"<!DOCTYPE html><html dir=""ltr"" lang=""en-IN"" cl...",Goal.com\n • SCORES\n • News\n • Tr...,2025,11,26,en,Getty Images Sport\nHarry Kane claims 'there's...
2,https://zeenews.india.com/railways/pune-metro-...,"<style type=""text/css""> \n #fv {\n\n positio...",हिन्दीEnglishमराठीবাংলাதமிழ்മലയാളംગુજરાતીతెలుగ...,2025,11,26,en,Pune Metro Phase-2 Extension: Cabinet Approves...
3,https://kz.kinoafisha.info/en/kostanay/movies/...,"<!DOCTYPE html>\n<html lang=""en"">\n<head>\n\t<...",Menu\nKinoafisha\nРусский English\nCancel\nSho...,2025,11,26,en,"Menu\nKinoafisha\nKostanay, KZ\nENG\nРусский\n..."
4,https://www.rawpixel.com/image/7384143/png-pap...,"<!DOCTYPE html><html lang=""en-US""><head><meta ...",rawpixel\nAll Media\nImages\nTemplates\nVideos...,2025,11,26,en,Edit ImageCropHeinSaveSaveEdit Imagetorn paper...
5,https://www.hawaiipublicradio.org/national-int...,"<!DOCTYPE html>\n<html class=""ArtP aside"" lang...",• Home\n • News\n • Local\n • Nationa...,2025,11,26,en,"Updated November 25, 2025 at 3:17 PM HST\nDemo..."
6,https://www.westernjournal.com/controversial-i...,"<!doctype html>\n<html lang=""en-US"">\n\t<head>...",Membership Log Out Manage\n • Premium\n • Ne...,2025,11,26,en,Controversial 'Islamic-Focused' Texas City Att...
7,https://www.actionforex.com/contributors/funda...,<!doctype html >\n\n<!--[if IE 8]> <html cl...,Facebook Mail RSS Twitter\n • Markets\n • ...,2025,11,26,en,Market Overview\nThe crypto market cap rose 0....
8,https://www.pearson.com/channels/physics/asset...,"<!DOCTYPE html><html lang=""en""><head><meta cha...",Skip to main content\nPearson+ LogoPearson+ Lo...,2025,11,26,en,Which information is necessary to determine th...
9,https://www.thehawk.in/news/crime/minor-sexual...,"<!DOCTYPE html><html lang=""en"" class=""__classN...",1. Home\n 2. >\n 3. Crime\nCrime\n\nMinor ...,2025,11,26,en,Bhopal/Mandideep: Hundreds of furious resident...


In [23]:
extract_text_with_docling(df.first()['html'])

/home/picatto/ascii/gen-ai/.pixi/envs/basic-research/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'Food price hike fears mount after supermarkets hit with property tax rise\nThe Government will introduce a new business rates ‘surtax’ on larger commercial properties worth £500,000 or more.\nSupermarkets are among those who have been dealt a blow in the Budget as the Chancellor confirmed they will face a jump in business rates amid warnings the move could push up food prices for shoppers.\nThe Government will introduce a new business rates “surtax”, which will mean many larger commercial properties worth £500,000 or more will face a higher rate of the property tax in order to help fund a permanent discount for smaller retailer, leisure and hospitality firms.\nIt marks an apparent U-turn after it was widely reported that supermarkets were expecting to be made exempt from the higher level after criticism from industry bosses.\nMarks & Spencer had earlier said proposals for the surtax would be “encouraging retailers to close larger high street stores”.\nThe Treasury said the move is des

In [35]:
df.select('uri').distinct().count()

5247908

In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = stopwords.words('english')
stop_words

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/picatto/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
@dataclass
class PreprocessingConfig:
    """Configuration for the preprocessing pipeline"""
    
    # Input/Output paths
    input_path: str = "/data/raid5/data/picatto/ascii/news/raw"
    output_path: str = "/data/raid5/data/picatto/ascii/news/clean"
    
    # Language filter
    target_language: str = "en"
    
    # Gopher Quality Filter thresholds (from datatrove)
    min_doc_words: int = 50
    max_doc_words: int = 100_000
    min_avg_word_length: float = 3.0
    max_avg_word_length: float = 10.0
    max_symbol_word_ratio: float = 0.1
    max_bullet_lines_ratio: float = 0.9
    max_ellipsis_lines_ratio: float = 0.3
    max_non_alpha_words_ratio: float = 0.8
    min_stop_words: int = 2
    stop_words: List[str] = field(default_factory=lambda: stopwords.words('english'))
    
    # Gopher Repetition Filter thresholds
    max_dup_line_char_frac: float = 0.3
    max_dup_line_frac: float = 0.3
    max_top_2_gram_frac: float = 0.2
    max_top_3_gram_frac: float = 0.18
    max_top_4_gram_frac: float = 0.16
    
    # C4 Quality Filter thresholds
    min_words_c4: int = 50
    max_words_c4: int = 100_000
    min_sentence_count: int = 3
    curly_bracket_ratio: float = 0.025  # max ratio of { } characters
    
    # Chunking config for RAG
    chunk_size: int = 512  # in words
    chunk_overlap: int = 50  # overlap in words
    
    # Processing
    batch_size: int = 1000
    num_partitions: int = 200

In [14]:
class QualityFilters:
    """
    Implements quality filters similar to datatrove:
    - GopherQualityFilter
    - GopherRepetitionFilter  
    - C4QualityFilter
    - FineWebQualityFilter
    """
    
    def __init__(self, config: PreprocessingConfig):
        self.config = config
    
    @staticmethod
    def count_words(text: str) -> int:
        """Count words in text"""
        if not text:
            return 0
        return len(text.split())
    
    @staticmethod
    def get_lines(text: str) -> List[str]:
        """Split text into lines"""
        if not text:
            return []
        return [line.strip() for line in text.split('\n') if line.strip()]
    
    @staticmethod
    def get_words(text: str) -> List[str]:
        """Get list of words"""
        if not text:
            return []
        return text.split()
    
    @staticmethod
    def get_ngrams(words: List[str], n: int) -> List[str]:
        """Generate n-grams from word list"""
        return [' '.join(words[i:i+n]) for i in range(len(words) - n + 1)]
    
    def gopher_quality_filter(self, text: str) -> Tuple[bool, str]:
        """
        Gopher Quality Filter - based on the Gopher paper quality heuristics
        Returns (passed, reason)
        """
        if not text:
            return False, "empty_text"
        
        words = self.get_words(text)
        n_words = len(words)
        
        # Word count checks
        if n_words < self.config.min_doc_words:
            return False, f"too_few_words:{n_words}"
        if n_words > self.config.max_doc_words:
            return False, f"too_many_words:{n_words}"
        
        # Average word length
        if words:
            avg_word_len = sum(len(w) for w in words) / len(words)
            if avg_word_len < self.config.min_avg_word_length:
                return False, f"avg_word_len_too_short:{avg_word_len:.2f}"
            if avg_word_len > self.config.max_avg_word_length:
                return False, f"avg_word_len_too_long:{avg_word_len:.2f}"
        
        # Symbol to word ratio (excessive symbols)
        symbol_chars = sum(1 for c in text if c in '#@<>{}[]|\\^~`')
        if n_words > 0 and symbol_chars / n_words > self.config.max_symbol_word_ratio:
            return False, "too_many_symbols"
        
        # Bullet lines ratio
        lines = self.get_lines(text)
        if lines:
            bullet_lines = sum(1 for line in lines if line.startswith(('•', '-', '*', '·', '►', '●')))
            if bullet_lines / len(lines) > self.config.max_bullet_lines_ratio:
                return False, "too_many_bullet_lines"
        
        # Ellipsis lines ratio
        if lines:
            ellipsis_lines = sum(1 for line in lines if line.endswith('...') or line.endswith('…'))
            if ellipsis_lines / len(lines) > self.config.max_ellipsis_lines_ratio:
                return False, "too_many_ellipsis_lines"
        
        # Non-alphabetic words ratio
        if words:
            non_alpha_words = sum(1 for w in words if not any(c.isalpha() for c in w))
            if non_alpha_words / n_words > self.config.max_non_alpha_words_ratio:
                return False, "too_many_non_alpha_words"
        
        # Stop words check (document should have some stop words to be natural)
        if words:
            stop_count = sum(1 for w in words if w.lower() in self.config.stop_words)
            if stop_count < self.config.min_stop_words:
                return False, f"too_few_stop_words:{stop_count}"
        
        return True, "passed"
    
    def gopher_repetition_filter(self, text: str) -> Tuple[bool, str]:
        """
        Gopher Repetition Filter - detect repetitive content
        Returns (passed, reason)
        """
        if not text:
            return False, "empty_text"
        
        lines = self.get_lines(text)
        words = self.get_words(text)
        
        if not lines or not words:
            return False, "no_content"
        
        # Duplicate line character fraction
        from collections import Counter
        line_counts = Counter(lines)
        dup_line_chars = sum(len(line) * (count - 1) for line, count in line_counts.items() if count > 1)
        total_chars = len(text)
        if total_chars > 0 and dup_line_chars / total_chars > self.config.max_dup_line_char_frac:
            return False, "duplicate_line_chars"
        
        # Duplicate line fraction
        if len(lines) > 0:
            dup_lines = len(lines) - len(set(lines))
            if dup_lines / len(lines) > self.config.max_dup_line_frac:
                return False, "duplicate_lines"
        
        # Top n-gram fractions
        if len(words) >= 2:
            ngram_2 = self.get_ngrams(words, 2)
            if ngram_2:
                top_2gram_count = Counter(ngram_2).most_common(1)[0][1] if ngram_2 else 0
                if top_2gram_count / len(ngram_2) > self.config.max_top_2_gram_frac:
                    return False, "repetitive_2grams"
        
        if len(words) >= 3:
            ngram_3 = self.get_ngrams(words, 3)
            if ngram_3:
                top_3gram_count = Counter(ngram_3).most_common(1)[0][1] if ngram_3 else 0
                if top_3gram_count / len(ngram_3) > self.config.max_top_3_gram_frac:
                    return False, "repetitive_3grams"
        
        if len(words) >= 4:
            ngram_4 = self.get_ngrams(words, 4)
            if ngram_4:
                top_4gram_count = Counter(ngram_4).most_common(1)[0][1] if ngram_4 else 0
                if top_4gram_count / len(ngram_4) > self.config.max_top_4_gram_frac:
                    return False, "repetitive_4grams"
        
        return True, "passed"
    
    def c4_quality_filter(self, text: str) -> Tuple[bool, str]:
        """
        C4 Quality Filter - based on C4 dataset heuristics
        Returns (passed, reason)
        """
        if not text:
            return False, "empty_text"
        
        words = self.get_words(text)
        n_words = len(words)
        
        # Word count
        if n_words < self.config.min_words_c4:
            return False, f"c4_too_few_words:{n_words}"
        if n_words > self.config.max_words_c4:
            return False, f"c4_too_many_words:{n_words}"
        
        # Sentence count (rough heuristic)
        sentences = re.split(r'[.!?]+', text)
        sentences = [s.strip() for s in sentences if s.strip()]
        if len(sentences) < self.config.min_sentence_count:
            return False, f"too_few_sentences:{len(sentences)}"
        
        # Curly bracket ratio (often indicates code/JSON)
        curly_count = text.count('{') + text.count('}')
        if len(text) > 0 and curly_count / len(text) > self.config.curly_bracket_ratio:
            return False, "too_many_curly_brackets"
        
        # Check for lorem ipsum / placeholder text
        if 'lorem ipsum' in text.lower():
            return False, "lorem_ipsum"
        
        # Check for common boilerplate phrases
        boilerplate_phrases = [
            'javascript is disabled',
            'enable javascript',
            'cookies are disabled',
            'please enable cookies',
            'access denied',
            'page not found',
            '404 error',
            'subscribe to our newsletter',
            'sign up for our newsletter',
        ]
        text_lower = text.lower()
        for phrase in boilerplate_phrases:
            if phrase in text_lower:
                return False, f"boilerplate:{phrase[:20]}"
        
        return True, "passed"
    
    def fineweb_quality_filter(self, text: str) -> Tuple[bool, str]:
        """
        FineWeb Quality Filter - additional heuristics from FineWeb paper
        Returns (passed, reason)
        """
        if not text:
            return False, "empty_text"
        
        lines = self.get_lines(text)
        
        # Check for lines that are just "Read more", "Continue reading", etc.
        nav_patterns = [
            r'^read more\.?$',
            r'^continue reading\.?$',
            r'^click here\.?$',
            r'^share this\.?$',
            r'^follow us\.?$',
            r'^subscribe\.?$',
            r'^sign in\.?$',
            r'^log in\.?$',
        ]
        nav_line_count = 0
        for line in lines:
            for pattern in nav_patterns:
                if re.match(pattern, line.lower().strip()):
                    nav_line_count += 1
                    break
        
        if len(lines) > 0 and nav_line_count / len(lines) > 0.3:
            return False, "too_many_nav_lines"
        
        # Check for excessive short lines (often menus/navigation)
        if lines:
            short_lines = sum(1 for line in lines if len(line.split()) < 3)
            if short_lines / len(lines) > 0.7:
                return False, "too_many_short_lines"
        
        return True, "passed"
    
    def apply_all_filters(self, text: str) -> Tuple[bool, str, Dict[str, str]]:
        """
        Apply all quality filters in sequence
        Returns (passed, final_reason, filter_results_dict)
        """
        filter_results = {}
        
        # Gopher Quality
        passed, reason = self.gopher_quality_filter(text)
        filter_results['gopher_quality'] = reason
        if not passed:
            return False, f"gopher_quality:{reason}", filter_results
        
        # Gopher Repetition
        passed, reason = self.gopher_repetition_filter(text)
        filter_results['gopher_repetition'] = reason
        if not passed:
            return False, f"gopher_repetition:{reason}", filter_results
        
        # C4 Quality
        passed, reason = self.c4_quality_filter(text)
        filter_results['c4_quality'] = reason
        if not passed:
            return False, f"c4_quality:{reason}", filter_results
        
        # FineWeb Quality
        passed, reason = self.fineweb_quality_filter(text)
        filter_results['fineweb_quality'] = reason
        if not passed:
            return False, f"fineweb_quality:{reason}", filter_results
        
        return True, "passed", filter_results

In [16]:
def clean_extracted_text(text: str) -> str:
    """
    Clean extracted text by removing common artifacts
    """
    if not text:
        return ""
    
    # Remove multiple newlines
    text = re.sub(r'\n{3,}', '\n\n', text)
    
    # Remove multiple spaces
    text = re.sub(r'[ \t]+', ' ', text)
    
    # Remove lines that are just navigation/boilerplate
    lines = text.split('\n')
    cleaned_lines = []
    
    skip_patterns = [
        r'^(home|menu|navigation|search|login|sign in|sign up|register)$',
        r'^(share|tweet|pin|email|print)$',
        r'^(facebook|twitter|instagram|linkedin|youtube)$',
        r'^\s*[|•·-]\s*$',  # Just separator characters
        r'^©.*\d{4}',  # Copyright lines
        r'^all rights reserved',
    ]
    
    for line in lines:
        line_stripped = line.strip().lower()
        skip = False
        for pattern in skip_patterns:
            if re.match(pattern, line_stripped, re.IGNORECASE):
                skip = True
                break
        if not skip:
            cleaned_lines.append(line)
    
    return '\n'.join(cleaned_lines).strip()

In [19]:
def process_partition_docling(iterator, config: PreprocessingConfig):
    """
    Process a partition of rows using Docling extraction and quality filters
    This runs on worker nodes via mapPartitions
    """
    # Initialize filters once per partition
    filters = QualityFilters(config)
    
    for row in iterator:
        try:
            html = row.html
            
            # Extract text
            # extracted_text = extract_text_with_trafilatura(html)
            extracted_text = extract_text_with_docling(html)
            
            if not extracted_text:
                continue
            
            # Clean text
            extracted_text = clean_extracted_text(extracted_text)
            
            if not extracted_text:
                continue
            
            # Apply quality filters
            passed, reason, _ = filters.apply_all_filters(extracted_text)
            
            # Generate doc ID
            doc_id = hashlib.sha256(row.uri.encode()).hexdigest()[:16]
            
            yield {
                'doc_id': doc_id,
                'uri': row.uri,
                'host': row.host,
                'extracted_text': extracted_text,
                'word_count': len(extracted_text.split()),
                'quality_passed': passed,
                'quality_reason': reason,
                'http_date': row.http_date,
                'year': row.year,
                'month': row.month,
                'day': row.day,
            }
            
        except Exception as e:
            logger.error(f"Error processing row: {e}")
            continue


In [24]:
config = PreprocessingConfig(
        input_path="/data/raid5/data/picatto/ascii/news/raw",
        output_path="/data/raid5/data/picatto/ascii/news/clean",
        target_language="en",
        chunk_size=512,
        chunk_overlap=50,
        num_partitions=200,
    )

In [34]:
df_test = df.sample(fraction=0.001, seed=3917)
df_test.show()
df_test.count()

+--------------------+--------------------+--------------------+--------------------+----+-----+---+---------+--------------------+
|                 uri|                host|                html|                text|year|month|day|main_lang|           http_date|
+--------------------+--------------------+--------------------+--------------------+----+-----+---+---------+--------------------+
|https://briefly.c...|       za.co.briefly|<!DOCTYPE html>\n...|Briefly.co.za Log...|2025|   11| 26|       en|2025-11-26T16:33:...|
|https://www.theha...|          in.thehawk|<!DOCTYPE html><h...|  1. Home\n  2. >...|2025|   11| 26|       en|2025-11-26T16:34:...|
|https://www.soomp...|          com.soompi|<!doctype html>\n...|  • EXPLORE\n    ...|2025|   11| 26|       en|2025-11-26T16:35:...|
|https://www.webpr...|      com.webpronews|            \r\n<...|webpronews.com\nW...|2025|   11| 26|       en|2025-11-26T16:53:...|
|https://www.goal....|            com.goal|<!DOCTYPE html><h...|Goal.com\n  

5213

In [28]:
from pyspark.sql.types import (
    StructType, StructField, StringType, IntegerType, BooleanType
)

In [29]:
def create_processing_schema() -> StructType:
    """Create schema for processed documents"""
    return StructType([
        # Document identifiers
        StructField("doc_id", StringType(), False),
        StructField("uri", StringType(), True),
        StructField("host", StringType(), True),
        
        # Extracted content
        StructField("extracted_text", StringType(), True),
        StructField("word_count", IntegerType(), True),
        
        # Quality filter results
        StructField("quality_passed", BooleanType(), True),
        StructField("quality_reason", StringType(), True),
        
        # Metadata
        StructField("http_date", StringType(), True),
        StructField("year", StringType(), True),
        StructField("month", StringType(), True),
        StructField("day", StringType(), True),
    ])

In [35]:
processed_rdd = df_test.rdd.mapPartitions(
        lambda it: process_partition_docling(it, config)
    )
processed_df = spark.createDataFrame(processed_rdd, schema=create_processing_schema())

In [36]:
processed_df.show()

+----------------+--------------------+----------------+--------------------+----------+--------------+--------------------+--------------------+----+-----+---+
|          doc_id|                 uri|            host|      extracted_text|word_count|quality_passed|      quality_reason|           http_date|year|month|day|
+----------------+--------------------+----------------+--------------------+----------+--------------+--------------------+--------------------+----+-----+---+
|64ea378606ba4a0d|https://briefly.c...|   za.co.briefly|Babes Wodumo Reme...|       571|          true|              passed|2025-11-26T16:33:...|2025|   11| 26|
|3dd722ac7b2f14b3|https://www.theha...|      in.thehawk|New Delhi: The Ca...|       362|          true|              passed|2025-11-26T16:34:...|2025|   11| 26|
|3aec2179969280a1|https://www.soomp...|      com.soompi|PLAVE And NMIXX E...|       450|          true|              passed|2025-11-26T16:35:...|2025|   11| 26|
|c1bad3255085225c|https://www.webp

In [38]:
processed_output

'/data/raid5/data/picatto/ascii/news/clean'

In [37]:
processed_df = processed_df.cache()
total_processed = processed_df.count()
passed_count = processed_df.filter(F.col("quality_passed") == True).count()
    
logger.info(f"Total processed: {total_processed}")
logger.info(f"Passed quality filters: {passed_count} ({passed_count/total_processed*100:.1f}%)")
   
logger.info("Filter rejection breakdown:")
processed_df.filter(F.col("quality_passed") == False) \
        .groupBy("quality_reason") \
        .count() \
        .orderBy(F.desc("count")) \
        .show(20, truncate=False)
    
    # ========================================
    # STAGE 2: Save Processed Documents (before chunking)
    # ========================================
logger.info("Saving processed documents...")
    
processed_output = f"{config.output_path}"
processed_df.filter(F.col("quality_passed") == True) \
        .write \
        .mode("overwrite") \
        .partitionBy("year", "month") \
        .parquet(processed_output)

logger.info(f"Saved processed documents to {processed_output}")

+--------------------------------------+-----+
|quality_reason                        |count|
+--------------------------------------+-----+
|gopher_repetition:duplicate_line_chars|215  |
|fineweb_quality:too_many_short_lines  |121  |
|gopher_quality:too_many_symbols       |69   |
|c4_quality:too_few_sentences:2        |37   |
|gopher_quality:too_few_words:13       |23   |
|gopher_repetition:duplicate_lines     |23   |
|c4_quality:too_few_sentences:1        |20   |
|gopher_quality:too_few_words:44       |14   |
|gopher_quality:too_few_words:48       |13   |
|gopher_quality:too_many_bullet_lines  |13   |
|gopher_quality:too_few_words:18       |13   |
|gopher_quality:too_few_words:39       |10   |
|gopher_quality:too_few_words:49       |10   |
|gopher_quality:too_few_stop_words:0   |10   |
|gopher_quality:too_few_words:42       |9    |
|gopher_quality:too_few_words:46       |9    |
|gopher_quality:too_few_words:47       |8    |
|gopher_quality:too_few_words:45       |8    |
|gopher_quali